In [ ]:
mport numpy as np

# Example packet delays (in ms)
packet_delays = [10, 20, 5, 15, 25]

# Objective function: minimize total completion time for given packet order
def objective(order):
    total_time = 0
    current_time = 0
    for i in order:
        current_time += packet_delays[i]
        total_time += current_time
    return total_time

# Generate a random solution (permutation of packets)
def random_solution(n):
    solution = np.arange(n)
    np.random.shuffle(solution)
    return solution

# Levy flight step (for continuous problems, adapted for permutation with swap)
def levy_flight(solution):
    n = len(solution)
    # Swap two random positions
    i, j = np.random.choice(n, 2, replace=False)
    new_solution = solution.copy()
    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution

def cuckoo_search(n_packets, n_nests=15, n_iterations=100, pa=0.25):
    # Initialize nests with random solutions
    nests = [random_solution(n_packets) for _ in range(n_nests)]
    fitness = np.array([objective(nest) for nest in nests])
   
    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx].copy()
    best_fitness = fitness[best_idx]
   
    for iter in range(n_iterations):
        # Generate new solutions by Levy flights
        for i in range(n_nests):
            new_nest = levy_flight(nests[i])
            new_fitness = objective(new_nest)
           
            # Greedy selection: keep better solutions
            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness
               
                # Update global best
                if new_fitness < best_fitness:
                    best_fitness = new_fitness
                    best_nest = new_nest.copy()
       
        # Abandon some nests and generate new ones
        for i in range(n_nests):
            if np.random.rand() < pa:
                nests[i] = random_solution(n_packets)
                fitness[i] = objective(nests[i])
               
                # Update global best
                if fitness[i] < best_fitness:
                    best_fitness = fitness[i]
                    best_nest = nests[i].copy()
       
        if iter % 10 == 0 or iter == n_iterations - 1:
            print(f"Iteration {iter+1}/{n_iterations}, Best fitness: {best_fitness}")
   
    return best_nest, best_fitness

# Run the Cuckoo Search optimization
best_order, best_cost = cuckoo_search(len(packet_delays))
print("\nOptimal packet order (indices):", best_order)
print("Minimum total delay:", best_cost)
